In [6]:
import numpy as np
import math
import scipy.sparse as sp
import random

In [7]:
def CompleteEdges(events,edges):
    for e1 in events:
        for e2 in events:
            for e3 in events:
                if (e1,e2) in edges:
                    if (e2,e3) in edges:
                        if (e1,e3) not in edges:
                            edges.add((e1,e3))
    return edges

In [8]:
def GetSerialExtensions(prefix, leftevents, edges):
    serialpatterns = list([])
    if (not leftevents):
        serialpatterns.append(prefix)
    else:
        for evnt in leftevents:
            flag = 1
            for prevnt in prefix:
                if (evnt,prevnt) in edges:
                    flag = 0
                    break
            if flag == 1:
                serialpatterns += GetSerialExtensions(prefix+[evnt], leftevents-{evnt}, edges)
    return serialpatterns

In [9]:
details = ""
patterns = list([])
serialpatterns = list([])
events = []
Dataspecs = 'Dataspecs1'
with open (Dataspecs+'.txt','r') as f:
    for line in f:
        pattern = line[:-1]
        print(pattern)
        details += pattern + '\n'
        entry = pattern.split(':')
        events.append(entry[0].split(','))
        edges = entry[1].split(',')
        edgelist = set()
        for edge in edges:
            edge_entry = edge.split('>')
            edgelist.add((int(edge_entry[0]),int(edge_entry[1])))
            # edgelist.add(tuple(edge.split('>')))
        edgelist = CompleteEdges(range(len(events)),edgelist)
        details += str(edgelist) + '\n'
        print(edgelist)
        serialpatterns.append(GetSerialExtensions([],set(range(len(events[-1]))),edgelist))
        patterns.append(events[-1])

evnt_serialpatterns = []
for k in range(len(serialpatterns)):
    evnt_serialpatterns.append([])
    details += '\n'
    for pattern in serialpatterns[k]:
        evnt_pattern = []
        for evnt_id in pattern:
            evnt_pattern.append(str(events[k][evnt_id]))
        evnt_serialpatterns[-1].append(evnt_pattern)
        details += str(evnt_pattern) + '\n'
print(details)

1,1,2,2,1,2:0>1,0>2,1>3,2>3,1>4,2>4,3>5,4>5
{(0, 1), (2, 4), (1, 4), (2, 3), (0, 2), (4, 5), (1, 3), (3, 5)}
3,4,5,3,4,5:0>1,0>2,1>3,1>4,2>5
{(0, 1), (1, 4), (0, 2), (2, 5), (1, 3)}
1,1,2,2,1,2:0>1,0>2,1>3,2>3,1>4,2>4,3>5,4>5
{(0, 1), (2, 4), (1, 4), (2, 3), (0, 2), (4, 5), (1, 3), (3, 5)}
3,4,5,3,4,5:0>1,0>2,1>3,1>4,2>5
{(0, 1), (1, 4), (0, 2), (2, 5), (1, 3)}

['1', '1', '2', '2', '1', '2']
['1', '1', '2', '1', '2', '2']
['1', '2', '1', '2', '1', '2']
['1', '2', '1', '1', '2', '2']

['3', '4', '5', '3', '4', '5']
['3', '4', '5', '3', '5', '4']
['3', '4', '5', '4', '3', '5']
['3', '4', '5', '4', '5', '3']
['3', '4', '5', '5', '3', '4']
['3', '4', '5', '5', '4', '3']
['3', '4', '3', '5', '4', '5']
['3', '4', '3', '5', '5', '4']
['3', '4', '3', '4', '5', '5']
['3', '4', '4', '5', '3', '5']
['3', '4', '4', '5', '5', '3']
['3', '4', '4', '3', '5', '5']
['3', '5', '4', '3', '4', '5']
['3', '5', '4', '3', '5', '4']
['3', '5', '4', '4', '3', '5']
['3', '5', '4', '4', '5', '3']
['3', '5', '4'

In [10]:
M = 60
T = 10000

N = len(patterns)


eta = 0.5
p = 0.03
rho = 0.01


N_set = 10
avg_episodeevnts = 0
avg_noiseevnts = 0
avg_noiseratio = 0
for n_set in range(N_set):
    data_stream_mat = sp.lil_matrix((M,T))

    print("Dataset: ", n_set)
    details += "Dataset: " + str(n_set) +'\n'

    alph_ptrn = set()
    for ep in patterns:
        for ev in ep:
            if ev not in alph_ptrn:
                alph_ptrn.add(ev)

    episodeevntcount = 0
    for k in range(N):
        t = 0
        e = 0
        count = 0
        while(1):
            if e == 0:
                count += 1
                t += np.random.geometric(p)
                l = random.choice(range(len(evnt_serialpatterns[k])))
            else:
                t += np.random.geometric(eta)
            if t >= T:
                break
            data_stream_mat[int(evnt_serialpatterns[k][l][e]),t] = 1
            episodeevntcount += 1
            e += 1
            e %= len(patterns[k])
        print("Episode", k+1 , "occurrences: ", count)
        details += "Episode " + str(k+1) + "occurrences: " + str(count) + '\n'
    
    noiseevntcount = 0
    for k in range(M):
        t = 0
        if str(k) in alph_ptrn:
            par = rho/5
        else:
            par = rho
        while(1):
            t += np.random.geometric(par)
            if t>= T:
                break
            data_stream_mat[k,t] = 1
            noiseevntcount += 1

    parameters = './' + Dataspecs + '_Dataset_M='+ str(M) + '_N=' + str(N) + '_rho=' + str(rho) + '_p=' + str(p) + '_eta=' + str(eta) + '_T=' + str(T)
    name = parameters +  '_evnt_strm_set=' + str(n_set) +'.txt'
    with open(name,'w') as f:
        for t in range(T):
            tt = str(int(t))
            for k in range(M):
                if data_stream_mat[k,t] == 1:
                    if k == 0:
                        f.write(str(M) + ',' + tt + '\n')
                    else:
                        f.write(str(k) + ',' + tt + '\n')
    print("Episode events: ", episodeevntcount)
    print("Noise events: ", noiseevntcount)
    noiseratio = noiseevntcount/(episodeevntcount+noiseevntcount)
    print("Noise ratio: ", noiseratio)
    
    avg_episodeevnts += episodeevntcount
    avg_noiseevnts += noiseevntcount
    avg_noiseratio += noiseratio
    
    details += 'Episode events: ' + str(episodeevntcount) + '\n' + 'Noise events: ' + str(noiseevntcount) + '\n' + 'Noise ratio: ' + str(noiseratio) + '\n'

avg_episodeevnts = avg_episodeevnts/N_set
avg_noiseevnts = avg_noiseevnts/N_set
avg_noiseratio = avg_noiseratio/N_set

details += '\n\n\n' + 'Average episode events: ' + str(avg_episodeevnts) + '\n' + 'Average noise events: ' + str(avg_noiseevnts) + '\n' + 'Average noise ratio: ' + str(avg_noiseratio) + '\n'

with open(parameters + '.txt', 'w') as f:
    f.write(details)
print("Done")

Dataset:  0
Episode 1 occurrences:  224
Episode 2 occurrences:  231
Episode events:  2718
Noise events:  5729
Noise ratio:  0.6782289570261631
Dataset:  1
Episode 1 occurrences:  220
Episode 2 occurrences:  235
Episode events:  2719
Noise events:  5582
Noise ratio:  0.6724491025177689
Dataset:  2
Episode 1 occurrences:  226
Episode 2 occurrences:  223
Episode events:  2682
Noise events:  5583
Noise ratio:  0.6754990925589837
Dataset:  3
Episode 1 occurrences:  234
Episode 2 occurrences:  241
Episode events:  2838
Noise events:  5648
Noise ratio:  0.6655668159321235
Dataset:  4
Episode 1 occurrences:  228
Episode 2 occurrences:  220
Episode events:  2676
Noise events:  5521
Noise ratio:  0.6735390996706112
Dataset:  5
Episode 1 occurrences:  244
Episode 2 occurrences:  233
Episode events:  2854
Noise events:  5665
Noise ratio:  0.6649841530696091
Dataset:  6
Episode 1 occurrences:  248
Episode 2 occurrences:  230
Episode events:  2856
Noise events:  5564
Noise ratio:  0.6608076009501188